In [67]:
pip install transformers sentence-transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [68]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import datetime
import random
import numpy as np
import pandas as pd

In [69]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [70]:
# Load the English version of the STSB dataset
dataset = load_dataset("stsb_multi_mt", "en")
print(dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5749
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})


In [71]:
len(dataset['train'])

5749

In [72]:
print("A sample from the STSB dataset's training split:")
print(dataset['train'][98])

A sample from the STSB dataset's training split:
{'sentence1': 'A man is slicing potatoes.', 'sentence2': 'A woman is peeling potato.', 'similarity_score': 2.200000047683716}


In [73]:
df = pd.read_csv('/kaggle/input/eng-train2/eng_train.csv')

In [74]:
df.head()

,PairID,Text,Score
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0
3,ENG-train-0003,If he is good looking and has a good personali...,1.0
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0


In [75]:
# Split the 'Text' column and create a DataFrame from the resulting list of lists
split_text = df['Text'].apply(lambda x: x.split('\n', 1)).to_list()
split_text_df = pd.DataFrame(split_text, columns=['sentence1', 'sentence2'])

# Assign the columns from the split_text_df to the original DataFrame (df)
df[['sentence1', 'sentence2']] = split_text_df[['sentence1', 'sentence2']]


In [76]:
df.head()

,PairID,Text,Score,sentence1,sentence2
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"It that happens, just pull the plug.","if that ever happens, just pull the plug."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,A black dog running through water.,A black dog is running through some water.
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,I've been searchingthe entire abbey for you.,I'm looking for you all over the abbey.
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,If he is good looking and has a good personali...,"If he's good looking, and a good personality, ..."
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"She does not hate you, she is just annoyed wit...","She doesn't hate you, she is just annoyed."


In [77]:
df.drop('PairID', axis = 1, inplace = True)
df.drop('Text', axis = 1, inplace = True)

In [78]:
df['similarity_score'] = df['Score'] * 5.0
df.drop('Score', axis=1, inplace = True)

In [79]:
df.head()
print(len(df))

5500


In [80]:
from sklearn.model_selection import train_test_split
train_df, dev_df = train_test_split(df, test_size=0.1, random_state = 1)

In [81]:
print(train_df.shape)
print(dev_df.shape)
print(len(dataset['train']))
print(dataset['train'][98])

(4950, 3)
(550, 3)
5749
{'sentence1': 'A man is slicing potatoes.', 'sentence2': 'A woman is peeling potato.', 'similarity_score': 2.200000047683716}


In [82]:
# sf=pd.read_csv('/kaggle/input/ansdata/eng_dev_with_labels.csv')

# split_text = sf['Text'].apply(lambda x: x.split('\n', 1)).to_list()
# split_text_sf = pd.DataFrame(split_text, columns=['sentence1', 'sentence2'])

# # Assign the columns from the split_text_df to the original DataFrame (df)
# sf[['sentence1', 'sentence2']] = split_text_df[['sentence1', 'sentence2']]

# sf.drop('PairID', axis = 1, inplace = True)
# sf.drop('Text', axis = 1, inplace = True)

# sf['similarity_score'] = sf['Score'] * 5.0
# sf.drop('Score', axis=1, inplace = True)

# sf.head()
# sf.shape

In [83]:
# kf=[dev_df,sf]
# dev_df = pd.concat(kf)
# dev_df.shape

In [84]:
combined  = pd.read_csv('/kaggle/input/d/sajid064/combined/combined.csv')
train_com_df, dev_com_df = train_test_split(combined, test_size=0.1, random_state = 1)

In [85]:
print(train_com_df.shape)
print(dev_com_df.shape)

(18362, 3)
(2041, 3)


In [86]:
print(dataset['train'][0]['sentence1'])

baal1 = list(train_df['sentence1'])
baal2 = list(train_df['sentence2'])
baal3 = list(train_df['similarity_score'])

baal11 = list(train_com_df['sentence1'])
baal22 = list(train_com_df['sentence2'])
baal33 = list(train_com_df['similarity_score'])

baal1.extend(baal11)
baal2.extend(baal22)
baal3.extend(baal33)
#updated_dataset = dataset['train'].map(lambda baal : {'sentence1': + baal1['baal']})

print(len(baal1))

A plane is taking off.
23312


In [87]:
from datasets import DatasetDict, Dataset

dataset['train'][0]['sentence1']

#baal1 = list(train_df['sentence1'])
#baal2 = list(train_df['sentence2'])
#baal3 = list(train_df['similarity_score'])

#updated_train_data
dataset2 = dataset.copy()
#print(len(dataset['train']))
# Create a new list of dictionaries with updated values
updated_train_data = []
#print(len(updated_train_data))

updated_train_data = [{'sentence1': s1, 'sentence2': s2, 'similarity_score': score} for s1, s2, score in zip(baal1, baal2, baal3)]

updated_train_data1 = [dataset['train'][i] for i in range(len(dataset['train']))]
print(len(updated_train_data1))

updated_train_data1.extend(updated_train_data)
# Assign the updated list back to dataset['train']

dataset2['train'] = updated_train_data1
print(len(dataset2['train']))
#print(len(updated_train_data))

#for i in range(0, 5749):
#    dataset2['train'][i] = dataset['train'][i].copy()

dataset['train'] = dataset2['train'].copy()

print(len(dataset['train']))

5749
29061
29061


In [88]:
baal1 = list(dev_df['sentence1'])
baal2 = list(dev_df['sentence2'])
baal3 = list(dev_df['similarity_score'])

baal11 = list(dev_com_df['sentence1'])
baal22 = list(dev_com_df['sentence2'])
baal33 = list(dev_com_df['similarity_score'])

baal1.extend(baal11)
baal2.extend(baal22)
baal3.extend(baal33)


#print(len(baal1))
updated_dev_data = []
# Create a new list of dictionaries with updated values
updated_dev_data = [{'sentence1': s1, 'sentence2': s2, 'similarity_score': score} for s1, s2, score in zip(baal1, baal2, baal3)]
updated_dev_data1 = [dataset['dev'][i] for i in range(len(dataset['dev']))]

#print(len(updated_dev_data))
#print(len(updated_dev_data1))

updated_dev_data1.extend(updated_dev_data)
print(len(updated_dev_data1))
# Assign the updated list back to dataset['train']

dataset2['dev'] = updated_dev_data1
#print(len(updated_dev_data))
#print(len(dataset['dev']))
#for i in range(0,1500):
#    dataset2['dev'][i] = dataset['dev'][i].copy()

dataset['dev'] = dataset2['dev'].copy()
    
print(len(dataset['dev']))

4091
4091


In [89]:
# You can use larger variants of the model, here we're using the base model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [90]:
class STSBDataset(torch.utils.data.Dataset):

    def __init__(self, dataset):
        # Normalize the similarity scores in the dataset
        similarity_scores = [i['similarity_score'] for i in dataset]
        self.normalized_similarity_scores = [i/5.0 for i in similarity_scores]
        self.first_sentences = [i['sentence1'] for i in dataset]
        self.second_sentences = [i['sentence2'] for i in dataset]
        self.concatenated_sentences = [[str(x), str(y)] for x,y in   zip(self.first_sentences, self.second_sentences)]

    def __len__(self):
        return len(self.concatenated_sentences)

    def get_batch_labels(self, idx):
        return torch.tensor(self.normalized_similarity_scores[idx])

    def get_batch_texts(self, idx):
        return tokenizer(self.concatenated_sentences[idx], padding='max_length', max_length=128, truncation=True, return_tensors="pt")

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y



def collate_fn(texts):
    input_ids = texts['input_ids']
    attention_masks = texts['attention_mask']
    features = [{'input_ids': input_id, 'attention_mask': attention_mask}
                for input_id, attention_mask in zip(input_ids, attention_masks)]
    return features

In [91]:
class BertForSTS(torch.nn.Module):

    def __init__(self):
        super(BertForSTS, self).__init__()
        self.bert = models.Transformer('bert-base-uncased', max_seq_length=128)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])

    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output

In [92]:
# Instantiate the model and move it to GPU
model = BertForSTS()
model.to(device)

BertForSTS(
  (bert): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (pooling_layer): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (sts_bert): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
)

In [93]:
class CosineSimilarityLoss(torch.nn.Module):

    def __init__(self,  loss_fn=torch.nn.MSELoss(), transform_fn=torch.nn.Identity()):
        super(CosineSimilarityLoss, self).__init__()
        self.loss_fn = loss_fn
        self.transform_fn = transform_fn
        self.cos_similarity = torch.nn.CosineSimilarity(dim=1)

    def forward(self, inputs, labels):
        emb_1 = torch.stack([inp[0] for inp in inputs])
        emb_2 = torch.stack([inp[1] for inp in inputs])
        outputs = self.transform_fn(self.cos_similarity(emb_1, emb_2))
        return self.loss_fn(outputs, labels.squeeze())

In [94]:
train_ds = STSBDataset(dataset['train'])
val_ds = STSBDataset(dataset['dev'])

# Create a 90-10 train-validation split.
train_size = len(train_ds)
val_size = len(val_ds)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

29,061 training samples
4,091 validation samples


In [95]:
train_ds[0]

({'input_ids': tensor([[ 101, 1037, 4946, 2003, 2635, 2125, 1012,  102,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0],
         [ 101, 2019, 2250, 4946, 2003, 2635, 2125, 1012,  102,    0,    0,    0,
             0,    0,    

In [96]:
batch_size = 8

train_dataloader = DataLoader(
            train_ds,  # The training samples.
            num_workers = 4,
            batch_size = batch_size, # Use this batch size.
            shuffle=True # Select samples randomly for each batch
        )

validation_dataloader = DataLoader(
            val_ds,
            num_workers = 4,
            batch_size = batch_size # Use the same batch size
        )

In [97]:
optimizer = AdamW(model.parameters(),
                  lr = 9e-6)
epochs = 12
# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)



In [98]:
# Define format_time function
import time

def format_time(elapsed):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(time.strftime("%H:%M:%S", time.gmtime(elapsed_rounded)))

In [ ]:
def train():
  seed_val = 42
  criterion = CosineSimilarityLoss()
  criterion = criterion.cuda()
  random.seed(seed_val)
  torch.manual_seed(seed_val)
  # We'll store a number of quantities such as training and validation loss,
  # validation accuracy, and timings.
  training_stats = []
  total_t0 = time.time()
  for epoch_i in range(0, epochs):
      t0 = time.time()
      total_train_loss = 0
      model.train()
      # For each batch of training data...
      for train_data, train_label in tqdm(train_dataloader):
          train_data['input_ids'] = train_data['input_ids'].to(device)
          train_data['attention_mask'] = train_data['attention_mask'].to(device)
          train_data = collate_fn(train_data)
          model.zero_grad()
          output = [model(feature) for feature in train_data]
          loss = criterion(output, train_label.to(device))
          total_train_loss += loss.item()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)
      t0 = time.time()
      model.eval()
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0
      # Evaluate data for one epoch
      for val_data, val_label in tqdm(validation_dataloader):
          val_data['input_ids'] = val_data['input_ids'].to(device)
          val_data['attention_mask'] = val_data['attention_mask'].to(device)
          val_data = collate_fn(val_data)
          with torch.no_grad():
              output = [model(feature) for feature in val_data]
          loss = criterion(output, val_label.to(device))
          total_eval_loss += loss.item()
      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )
  return model, training_stats

# Launch the training
model, training_stats = train()


 36%|███▋      | 186/512 [00:19<00:32,  9.91it/s]

In [ ]:
# Create a DataFrame from our training statistics
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index
df_stats = df_stats.set_index('epoch')

# Display the table
df_stats

In [ ]:
# load the test set
test_dataset = load_dataset("stsb_multi_mt", name="en", split="test")


test_df = pd.read_csv('/kaggle/input/eng-train2/eng_dev.csv')
#print(test_dataset)

test_df[['sentence1', 'sentence2']] = test_df['Text'].apply(lambda x: x.split('\n', 1)).to_list()
test_df.drop('Text', axis=1, inplace=True)
#dev_df.drop('PairID', axis=1, inplace=True)
test_df['similarity_score'] = 0.0
baal1 = list(test_df['sentence1'])
baal2 = list(test_df['sentence2'])
baal3 = list(test_df['similarity_score'])
# Create a new list of dictionaries with updated values
updated_train_data = [{'sentence1': s1, 'sentence2': s2, 'similarity_score': s3} for s1, s2, s3 in zip(baal1, baal2, baal3)]

# Assign the updated list back to dataset['train']
test_dataset = updated_train_data


# Prepare the data
first_sent = [i['sentence1'] for i in test_dataset]
second_sent = [i['sentence2'] for i in test_dataset]
full_text = [[str(x), str(y)] for x,y in zip(first_sent, second_sent)]

print(len(full_text))

model.eval()

def predict_similarity(sentence_pair):
  test_input = tokenizer(sentence_pair, padding='max_length', max_length = 128, truncation=True, return_tensors="pt").to(device)
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
  return sim

In [ ]:
example_1 = full_text[0]
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

In [ ]:
scor_list = []
for i in full_text:
    score = round(predict_similarity(i), 2)
    if(score<0):
        score = score * (-1)
    print(score)
    scor_list.append(score)

In [ ]:
test_df.drop('sentence1', axis=1, inplace=True)
test_df.head()

In [ ]:
test_df.drop('sentence2', axis=1, inplace=True)
test_df.drop('similarity_score', axis=1, inplace=True)

In [ ]:
test_df.head()

In [ ]:
test_df['Pred_Score'] = scor_list
test_df.head()

In [ ]:
test_df.to_csv('pred_eng_a.csv', index=False)